## Problem 4
### 4.1 
#### Report word-tag counts, tag unigram counts, and tag bigram counts. Let’s denote these by C (wi ,ti), C (ti), and C (ti−1 ,ti) 

In [2]:
#read files and remove space lines
import glob
read_files = glob.glob("C:\\Users\\81468\\OneDrive\\Documents\\CS6120\\homework\\a1\\a1_4\\brown\\*")
train = []
for i in range(len(read_files)):
    with open(read_files[i], "rb") as infile:
        for line in infile:
            if not line.isspace():
                train.append(line.decode('utf-8'))

In [3]:
# add <s>/START <e>/END, replace duplicate spaces and \n to single space
import re
for i in range(len(train)):
        train[i] = '<s>/START %s <e>/END' %train[i]
        train[i] = re.sub(r'[\s]',' ',train[i]) 
        train[i] = re.sub(r' {2,}',' ',train[i]) 

In [4]:
train_raw = ' '.join(train) #whole string
train_wt_l = train_raw.split(' ') #list of word_tag

In [5]:
def count(s):
    c=dict.fromkeys(set(s),0)
    for i in range(0,len(s)):
        c[s[i]] = c[s[i]]+1
    return c

In [6]:
word = [] #list of words
for i in range(len(train_wt_l)):
    word.append(train_wt_l[i].rsplit('/', 1)[0])

In [7]:
word_c = count(word) #count distinct word

In [8]:
#replace words occuring less than 5 times with UNK in word list
word_rep = []
for i in range(len(word)):
    if word_c[word[i]] <= 5:
        word_rep.append('<UNK>')
    else:
        word_rep.append(word[i])

In [9]:
#replace words occuring less than 5 times with UNK in word_tag list
word_tag_rep = []
for i in range(len(train_wt_l)):
    if word_c[word[i]] <= 5:
        word_tag_rep.append('<UNK>/%s' % train_wt_l[i].rsplit('/', 1)[1])
    else:
        word_tag_rep.append(train_wt_l[i])

In [10]:
#count replace word_tag list
word_tag_rep_count = count(word_tag_rep)

In [11]:
#count replaced word list
word_rep_count = count(word_rep)

In [12]:
tag = [] #list of tags
for i in range(len(train_wt_l)):
    tag.append(train_wt_l[i].rsplit('/', 1)[1])

In [13]:
#count tag
tag_count = count(tag)

In [15]:
#count bi_tag{'tag1 tag2':c1, 'tag2 tag3':c2,...}
bi_tag = []
for i in range(len(tag)-1):
    bi_tag.append(tag[i]+' '+tag[i+1])
bi_tag_count = count(bi_tag)

In [18]:
#report files
with open('word_tag_counts.txt', 'w') as f:
    for key, value in word_tag_rep_count.items():
        f.write('%s:%s\n' % (key, value))

with open('tag_counts.txt', 'w') as f:
    for key, value in tag_count.items():
        f.write('%s:%s\n' % (key, value))

with open('bi_tag_counts.txt', 'w') as f:
    for key, value in bi_tag_count.items():
        f.write('%s:%s\n' % (key, value))

### 4.2 
####  transition probabilities of the training set

In [19]:
V = len(tag_count)
uniq = list(set(tag_count.keys()))
prob_tag_smooth = {}
for i in range(V):
    for j in range(V):
        k2 = uniq[i]+' '+uniq[j]
        if k2 in bi_tag_count:
            prob_tag_smooth[k2] = (bi_tag_count[k2]+0.1)/(tag_count[uniq[i]]+0.1*V)
        else:
            prob_tag_smooth[k2] = 0.1/(tag_count[uniq[i]] + 0.1*V)

In [20]:
with open('bi_tag_prob.txt', 'w') as f:
    for key, value in prob_tag_smooth.items():
        f.write('%s:%s\n' % (key, value))

In [21]:
#smoothed transition probability in format {tag1:{{tag1:p1},{tag2,p2},...},tag2:{{tag1:p3},{tag2:p4},...},...}
transition = {}
for t in tag_count:
    transition[t] = {}
    for pt in prob_tag_smooth:
        if pt.split(' ')[0] == t:
            transition[t][pt.split(' ')[1]] = prob_tag_smooth[pt]

In [22]:
#unsmoothed transition probability
prob_tag = {}
for k2 in bi_tag_count:
            prob_tag[k2] = bi_tag_count[k2]/tag_count[k2.split(' ')[0]]

### 4.3
#### emission probabilities of the training set 

In [23]:
#change the format to {tag1:{word1:count1,word2:count2,...},tag2:{word1:count1,word3:count3,...},...}
word_tag_rep_count_2 = dict.fromkeys(tag_count,{})
for t in word_tag_rep_count_2:
    word_tag_rep_count_2[t]={}
    for wt in word_tag_rep_count.keys():
        if wt.rsplit('/',1)[1] == t:
            word_tag_rep_count_2[t][wt.rsplit('/',1)[0]] = word_tag_rep_count[wt]

In [24]:
#smoothed emission probability: in format {tag1:{word1:p1,word2:p2,...},tag2:{word1:p1,word3:p,...},...}
V_w = len(word_rep_count) #number of unique words (including <UNK>)
uniq_w = list(set(word_rep_count.keys())) #list of unique words (including <UNK>)
emission = {}
for t in word_tag_rep_count_2:
    emission[t] = {}
    for w in uniq_w:
        if w in word_tag_rep_count_2[t]:
            emission[t][w]=(word_tag_rep_count_2[t][w]+0.1)/(tag_count[t]+0.1*V_w)
        else:
            emission[t][w]= 0.1/(tag_count[t]+0.1*V_w)

In [25]:
#a different version for report
prob_word_tag_smoothed = {}
for t in emission:
    for w in emission[t]:
        prob_word_tag_smoothed['%s/%s' %(w,t)] = emission[t][w]

In [26]:
#report smoothed one
with open('word_tag_prob.txt', 'w') as f:
    for key, value in prob_word_tag_smoothed.items():
        f.write('%s:%s\n' % (key, value))

In [27]:
#unsmoothed prob_word_tag
prob_word_tag = {}
for wt in word_tag_rep_count:
    prob_word_tag[wt] = word_tag_rep_count[wt]/tag_count[wt.rsplit('/',1)[1]]

### 4.4
#### Generate 5 random sentences using the previously learned HMM. Output each sentence (with the POS tags) and its probability of being generated.

In [28]:
#generate sentences using unsmoothed emission probability and unsmoothed transition probability
import random
from functools import reduce
for k in range(0,5):
    sentence_tag =[]
    tag_p = []
    w = {} 
    s = 'START'
    while s != 'END':
        for bt in prob_tag:
            if bt.split(' ')[0] == s:
                w[bt] = prob_tag[bt]
        v = random.choice(list(w.items()))
        sentence_tag.append(v[0])
        tag_p.append(v[1])
        w = {} 
        s = sentence_tag[-1].split(' ')[1]
    w_2 = {}
    sentence_w = []
    word_p = []
    for i in range(len(sentence_tag)-1):
        for wt in prob_word_tag:
            if wt.rsplit('/',1)[1] == sentence_tag[i].split(' ')[1]:
                w_2[wt] = prob_word_tag[wt]
        v = random.choice(list(w_2.items()))
        sentence_w.append(v[0].rsplit('/',1)[0])
        word_p.append(v[1])
        w_2 = {}
    sentence = ' '.join(sentence_w)
    p = reduce(lambda x, y: x*y, word_p) * reduce(lambda x, y: x*y, tag_p)
    print('<S_%s> %s' %(k+1,sentence))
    print('Prob = %s'% p)

<S_1> on till me is ' fear Bridge act Water out up Success account -- was yours '' <UNK> You Belgians believing dark <UNK> you the -- it No nineteenth Liberal Set reassuring The Greater Garibaldi's Americans If : ?
Prob = 3.1870591164904557e-163
<S_2> I'd been away .
Prob = 8.900890826495666e-10
<S_3> They'd easier : <UNK> injustice Though <UNK> depending does <UNK> <UNK> accruing Major State's `` Americans or President's <UNK> Republic each Sunday's 700 , arms ( John , more to be lighter ease out , trusts toward ) ? :
Prob = 5.027651129293066e-146
<S_4> downtown : Berlin or how The International , may second student could be better !
Prob = 5.0802085824224045e-43
<S_5> Ill. )
Prob = 7.958515235021437e-07


### 4.5
#### For each word in the test dataset, derive the most probable POS tag sequence using the Viterbi algorithm

In [29]:
#read test file and combine sentences into a list 
with open('Test_File.txt', "rb") as infile:
    data = infile.read().decode('utf-8').split('<EOS>')
for i in range(len(data)):
    data[i] = list(filter(None, data[i].split('\r\n')))

In [30]:
#use Viterbi algorithm to calculate the best path(most possible POS tag sequence)
bestpath = {}
bestpathprob = {}
bestpathpointer = {}
for i in range(len(data)-1):
    T = len(data[i])
    #initialize viterbi{N:{T:}}
    t = 1
    viterbi = {}
    backpointer = {}
    for s in tag_count:
        viterbi[s] = {}
        backpointer[s] = {}
        if data[i][t] in word_rep_count:
            viterbi[s][data[i][t]] = transition['START'][s] * emission[s][data[i][t]]
        else:
            viterbi[s][data[i][t]] = transition['START'][s] * emission[s]['<UNK>']
        backpointer[s][data[i][t]] = 0
    for t in range(2,T):
        for s in tag_count:
            viterbi[s][data[i][t]] = 0
            for s_0 in tag_count:
                if data[i][t] in word_rep_count:
                    new_p = viterbi[s_0][data[i][t-1]] * transition[s_0][s] * emission[s][data[i][t]]
                else:
                    new_p = viterbi[s_0][data[i][t-1]] * transition[s_0][s] * emission[s]['<UNK>']
                viterbi[s][data[i][t]]= max(viterbi[s][data[i][t]], new_p)
                if viterbi[s][data[i][t]] == new_p:
                    backpointer[s][data[i][t-1]] = s_0
    bestpathprob[data[i][0]] = 0
    for s in tag_count:
        bestpathprob[data[i][0]] = max(bestpathprob[data[i][0]], viterbi[s][data[i][T-1]])
        if bestpathprob[data[i][0]] == viterbi[s][data[i][T-1]]:
            bestpathpointer[data[i][0]] = s
        backpointer[s][data[i][T-1]] = bestpathpointer[data[i][0]]
    bestpath[data[i][0]] = backpointer[bestpathpointer[data[i][0]]]
    #print(i)

In [31]:
#report best path
with open('POS_tag_result.txt', 'w') as f:
    for key in bestpath:
        f.write('%s\n' % key)
        for nest_key, value in bestpath[key].items():
            f.write('%s, %s\n' % (nest_key, value))
        f.write('<EOS>\n')